# Lab 1 - Data Exploration and Analysis -- 2013/2014 CitiBike-NYC Data
**Michael Smith, Alex Frye, Chris Boomhower ----- 1/29/2017**

<img src="Citi-Bike.jpg" width="400">
<center>Image courtesy of http://newyorkeronthetown.com/, 2017</center>

### Business Understanding
***Describe the purpose of the data set you selected***

The data set selected by our group for this lab primarily consists of [Citi Bike trip history](https://www.citibikenyc.com/system-data) collected and released by NYC Bike Share, LLC and Jersey Bike Share, LLC under Citi Bike's [NYCBS Data Use Policy](https://www.citibikenyc.com/data-sharing-policy). Citi Bike is America's largest bike share program, with 10,000 bikes and 600 stations across Manhattan, Brooklyn, Queens, and Jersey City... 55 neighborhoods in all. As such, our data set's trip history includes all rental transactions conducted within the NYC Citi Bike system from July 1st, 2013 to February 28th, 2014. These transactions amount to <span style="color:red">*####*</span> trips within this time frame. The original data set includes 15 attributes. That being said, our team was able to derive <span style="color:red">*####*</span> more attributes from the original 15 as disussed in detail in the next section. Of particular note, however, we merged NYC weather data from the [Carbon Dioxide Information Analysis Center (CDIAC)](http://cdiac.ornl.gov/cgi-bin/broker?_PROGRAM=prog.climsite_daily.sas&_SERVICE=default&id=305801&_DEBUG=0) with the Citi Bike data to provide environmental insights into rental behavior as well.

The trip data was collected via Citi Bike's check-in/check-out system among <span style="color:red">*####*</span> of its stations in the NYC system as part of its transaction history log. While the non-publicized data likely includes further particulars such as rider payment details, the publicized data is anonymized to protect rider identity while simultaneously offering bike share transportation insights to urban developers, engineers, academics, statisticians, and other interested parties. The CDIAC data, however, was collected by the Department of Energy's Oak Ridge National Laboratory for research into global climate change. While basic weather conditions are recorded by CDIAC, as included in our fully merged data set, the organization also measures atmospheric carbon dioxide and other radiatively active gas levels to conduct their research efforts.

Our team has taken particular interest in this data set as some of our team members enjoy both recreational and commute cycling. By combining basic weather data with Citi Bike's trip data, we expect to be able to predict whether riders are more likely to be (or become) Citi Bike subscribers based on ride environmental conditions, the day of the week for his/her trip, trip start and end locations, the general time of day (i.e. morning, midday, afternoon, evening, night) of his/her trip, his/her age and gender, etc. Deeper analysis may even yield further insights, such as identifying gaps in station location, for example. Furthermore, quantifiable predictions such as a rider's age as a function of trip distance and duration given other factors would provide improved targeting to bike share marketing efforts in New York City. Likewise, trip duration could be predicted based on other attributes which would allow the company to promote recreational cycling via factor adjustments within its control. By leveraging some of the vast number of trip observations as training data and others as test data via randomized selection, we expect to be able to measure the effectiveness of our algorithms and models throughout the semester.

### Data Understanding
***Describe the meaning and type of data***

..Written response..

*First discuss the care needed to utilize this data set in a non-time-series fashion*

While attributes such as trip duration, start and end stations, bike ID, and basic rider details were collected and shared with the general public, care has been taken by Citi Bike to remove 

#### Compiling Multiple Data Sources

In [1]:
import os
import sys
import re
from geopy.distance import vincenty
import holidays
from datetime import datetime
from dateutil.parser import parse
import glob
import pandas as pd
import numpy as np

In [2]:
starttime = datetime.now()
print(starttime)

if os.path.isfile("Compiled Data/dataset1.csv"):
    print("Found the File!")
else:
    citiBikeDataDirectory = "Citi Bike Data"
    citiBikeDataFileNames =[
        "2013-07 - Citi Bike trip data - 1.csv",
        "2013-07 - Citi Bike trip data - 2.csv",
        "2013-08 - Citi Bike trip data - 1.csv",
        "2013-08 - Citi Bike trip data - 2.csv",
        "2013-09 - Citi Bike trip data - 1.csv",
        "2013-09 - Citi Bike trip data - 2.csv",
        "2013-10 - Citi Bike trip data - 1.csv",
        "2013-10 - Citi Bike trip data - 2.csv",
        "2013-11 - Citi Bike trip data - 1.csv",
        "2013-11 - Citi Bike trip data - 2.csv",
        "2013-12 - Citi Bike trip data.csv",
        "2014-01 - Citi Bike trip data.csv",
        "2014-02 - Citi Bike trip data.csv"
    ]

    weatherDataFile = "Weather Data/NY305801_9255_edited.txt"

    citiBikeDataRaw = []

    for file in citiBikeDataFileNames:
        print(file)
        filepath = citiBikeDataDirectory + "/" + file
        with open(filepath) as f:
            lines = f.read().splitlines()
            lines.pop(0)  # get rid of the first line that contains the column names
            for line in lines:
                line = line.replace('"', '')
                line = line.split(",")
                sLatLong = (line[5], line[6])
                eLatLong = (line[9], line[10])

                distance = vincenty(sLatLong, eLatLong).miles
                line.extend([distance])

                ## Monday       = 0
                ## Tuesday      = 1
                ## Wednesday    = 2
                ## Thursday     = 3
                ## Friday       = 4
                ## Saturday     = 5
                ## Sunday       = 6
                if parse(line[1]).weekday() == 0:
                    DayOfWeek = "Monday"
                elif parse(line[1]).weekday() == 1:
                    DayOfWeek = "Tuesday"
                elif parse(line[1]).weekday() == 2:
                    DayOfWeek = "Wednesday"
                elif parse(line[1]).weekday() == 3:
                    DayOfWeek = "Thursday"
                elif parse(line[1]).weekday() == 4:
                    DayOfWeek = "Friday"
                elif parse(line[1]).weekday() == 5:
                    DayOfWeek = "Saturday"
                else:
                    DayOfWeek = "Sunday"
                line.extend([DayOfWeek])

                ##Morning       5AM-10AM
                ##Midday        10AM-2PM
                ##Afternoon     2PM-5PM
                ##Evening       5PM-10PM
                ##Night         10PM-5AM

                if parse(line[1]).hour >= 5 and parse(line[1]).hour < 10:
                    TimeOfDay = 'Morning'
                elif parse(line[1]).hour >= 10 and parse(line[1]).hour < 14:
                    TimeOfDay = 'Midday'
                elif parse(line[1]).hour >= 14 and parse(line[1]).hour < 17:
                    TimeOfDay = 'Afternoon'
                elif parse(line[1]).hour >= 17 and parse(line[1]).hour < 22:
                    TimeOfDay = 'Evening'
                else:
                    TimeOfDay = 'Night'
                line.extend([TimeOfDay])

                ## 1 = Yes
                ## 0 = No
                if parse(line[1]) in holidays.UnitedStates():
                    holidayFlag = "1"
                else:
                    holidayFlag = "0"
                line.extend([holidayFlag])

                citiBikeDataRaw.append(line)
            del lines

    with open(weatherDataFile) as f:
        weatherDataRaw = f.read().splitlines()
        weatherDataRaw.pop(0)  # again, get rid of the column names
        for c in range(len(weatherDataRaw)):
            weatherDataRaw[c] = weatherDataRaw[c].split(",")
            # Adjust days and months to have a leading zero so we can capture all the data
            if len(weatherDataRaw[c][2]) < 2:
                weatherDataRaw[c][2] = "0" + weatherDataRaw[c][2]
            if len(weatherDataRaw[c][0]) < 2:
                weatherDataRaw[c][0] = "0" + weatherDataRaw[c][0]

    citiBikeData = []

    while (citiBikeDataRaw):
        instance = citiBikeDataRaw.pop()
        date = instance[1].split(" ")[0].split("-")  # uses the start date of the loan
        for record in weatherDataRaw:
            if (str(date[0]) == str(record[4]) and str(date[1]) == str(record[2]) and str(date[2]) == str(record[0])):
                instance.extend([record[5], record[6], record[7], record[8], record[9]])
                citiBikeData.append(instance)

    del citiBikeDataRaw
    del weatherDataRaw

    # Final Columns:
    #  0 tripduration
    #  1 starttime
    #  2 stoptime
    #  3 start station id
    #  4 start station name
    #  5 start station latitude
    #  6 start station longitude
    #  7 end station id
    #  8 end station name
    #  9 end station latitude
    # 10 end station longitude
    # 11 bikeid
    # 12 usertype
    # 13 birth year
    # 14 gender
    # 15 start/end station distance
    # 16 DayOfWeek
    # 17 TimeOfDay
    # 18 HolidayFlag
    # 19 PRCP
    # 20 SNOW
    # 21 TAVE
    # 22 TMAX
    # 23 TMIN

    maxLineCount = 250000
    lineCounter = 1
    fileCounter = 1
    outputDirectoryFilename = "Compiled Data/dataset"
    f = open(outputDirectoryFilename + str(fileCounter) + ".csv", "w")
    for line in citiBikeData:
        if lineCounter == 250000:
            print(f)
            f.close()
            lineCounter = 1
            fileCounter = fileCounter + 1
            f = open(outputDirectoryFilename + str(fileCounter) + ".csv", "w")
        f.write(",".join(map(str, line)) + "\n")
        lineCounter = lineCounter + 1

    del citiBikeData
        
endtime = datetime.now()
print("RunTime: ")
print(endtime-starttime)

2017-01-26 00:42:50.716567
Found the File!
RunTime: 
0:00:00.001501


#### Loading the Compiled Data from CSV

In [3]:
def reader(f, columns):
    d = pd.read_csv(f)
    d.columns = columns
    return d


path = r'Compiled Data'
all_files = glob.glob(os.path.join(path, "*.csv"))

columns = ["tripduration", "starttime", "stoptime", "start_station_id", "start_station_name", "start_station_latitude",
           "start_station_longitude", "end_station_id", "end_station_name", "end_station_latitude",
           "end_station_longitude", "bikeid", "usertype", "birth year", "gender", "LinearDistance", "DayOfWeek",
           "TimeOfDay", "HolidayFlag", "PRCP", "SNOW", "TAVE", "TMAX", "TMIN"]

CitiBikeDataCompiled = pd.concat([reader(f, columns) for f in all_files])

    #Replace '\N' Birth Years with Zero Values
CitiBikeDataCompiled["birth year"] = CitiBikeDataCompiled["birth year"].replace(r'\N','0')

# Convert Columns to Numerical Values
CitiBikeDataCompiled[['tripduration', 'bikeid', 'birth year', 'LinearDistance','PRCP', 'SNOW', 'TAVE', 'TMAX', 'TMIN']]\
    = CitiBikeDataCompiled[['tripduration', 'bikeid', 'birth year','LinearDistance', 'PRCP', 'SNOW', 'TAVE', 'TMAX',
                            'TMIN']].apply(pd.to_numeric)

# Convert Columns to Date Values
CitiBikeDataCompiled[['starttime', 'stoptime']] \
    = CitiBikeDataCompiled[['starttime', 'stoptime']].apply(pd.to_datetime)

    ## Compute Age: 0 Birth Year = 0 Age ELSE Compute Start Time Year Minus Birth Year
CitiBikeDataCompiled["Age"] = np.where(CitiBikeDataCompiled["birth year"]==0, 0,
                                       CitiBikeDataCompiled["starttime"].dt.year - CitiBikeDataCompiled["birth year"])

print(CitiBikeDataCompiled.head())

1
2
3
4
5
6
   tripduration           starttime            stoptime  start_station_id  \
0           308 2014-02-28 23:59:10 2014-03-01 00:04:18               353   
1           304 2014-02-28 23:58:17 2014-03-01 00:03:21               497   
2          1355 2014-02-28 23:57:55 2014-03-01 00:20:30               470   
3           848 2014-02-28 23:57:13 2014-03-01 00:11:21               498   
4           175 2014-02-28 23:57:12 2014-03-01 00:00:07               383   

           start_station_name  start_station_latitude  \
0  S Portland Ave & Hanson Pl               40.685396   
1          E 17 St & Broadway               40.737050   
2             W 20 St & 8 Ave               40.743453   
3          Broadway & W 32 St               40.748549   
4  Greenwich Ave & Charles St               40.735238   

   start_station_longitude  end_station_id       end_station_name  \
0               -73.974315             365  Fulton St & Grand Ave   
1               -73.990093             334  

### Data Quality

two attributes with large quantity of missing values: birth year, gender as this is optional information for non-subscribing customers. build plot for missing value count by user type to show this. Gender is already defaulted at "0" for not provided; we have defaulted our NAs to 0 for birth year and will need to account for these missing values cautiously in our models moving forward.

no duplicates

Discuss our computed distance, and how it may not accurately represent the "actual" trip the biker took. we do not have enough information to more accurately compute this distance  

search for outlier duration times small and large; outlier distances; age

### Visualize appropriate statistics

- provide panda .describe output and interpret
 
- group data by trip start and end, identify variance in trip durations for the same route. do we trust our distance calc?

- distribution of age

In [4]:
CitiBikeDataCompiled.describe()

,tripduration,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth year,gender,LinearDistance,HolidayFlag,PRCP,SNOW,TAVE,TMAX,TMIN,Age
count,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06,5.562293e+06
mean,8.788105e+02,4.392880e+02,4.073437e+01,-7.399097e+01,4.396748e+02,4.073407e+01,-7.399108e+01,1.768373e+04,1.733736e+03,1.081256e+00,1.121726e+00,2.460101e-02,6.233738e-02,3.806081e-02,6.169050e+01,6.851392e+01,5.437925e+01,3.296077e+01
std,3.996950e+03,3.418623e+02,1.985896e-02,1.237109e-02,3.467747e+02,1.987918e-02,1.246342e-02,1.761230e+03,6.475671e+02,5.653095e-01,8.459473e-01,1.549058e-01,2.016957e-01,4.336314e-01,1.675271e+01,1.730100e+01,1.647196e+01,1.609177e+01
min,6.000000e+01,7.200000e+01,4.068034e+01,-7.401713e+01,7.200000e+01,4.068034e+01,-7.401713e+01,1.452900e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e+01,1.700000e+01,4.000000e+00,0.000000e+00
25%,3.980000e+02,3.040000e+02,4.072066e+01,-7.400004e+01,3.030000e+02,4.072043e+01,-7.400026e+01,1.616700e+04,1.963000e+03,1.000000e+00,5.357136e-01,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01,5.600000e+01,4.300000e+01,2.600000e+01
50%,6.340000e+02,4.030000e+02,4.073625e+01,-7.399076e+01,4.020000e+02,4.073544e+01,-7.399076e+01,1.768200e+04,1.976000e+03,1.000000e+00,8.846575e-01,0.000000e+00,0.000000e+00,0.000000e+00,6.500000e+01,7.300000e+01,5.700000e+01,3.300000e+01
75%,1.042000e+03,4.860000e+02,4.075020e+01,-7.398195e+01,4.840000e+02,4.074972e+01,-7.398195e+01,1.917700e+04,1.983000e+03,1.000000e+00,1.465782e+00,0.000000e+00,1.000000e-02,0.000000e+00,7.500000e+01,8.200000e+01,6.800000e+01,4.300000e+01
max,6.250750e+06,3.002000e+03,4.077051e+01,-7.395005e+01,3.002000e+03,4.077051e+01,-7.395005e+01,2.154200e+04,1.997000e+03,2.000000e+00,6.491681e+00,1.000000e+00,1.980000e+00,1.100000e+01,9.000000e+01,9.800000e+01,8.300000e+01,1.150000e+02


### Visualize Interesting Attributes

- Trip Duration by day of week [Boxplot]

- Average Trip linear distance by day of week [Boxplot]

- Average Trip duration by start time of day (evening, morning, etc.)[Boxplot]

- Holidays with the largest bike traffic avg holiday compared to avg non holiday [Barchart]

- percentage of bike users that are subscribers vs non-subscribers [pie]

- age group / gender by avg trip duration [barplot] (note in interpretation to discuss missing values for nonsubscribing members)

### Visualize Relationships Between Attributes

- [Parallel coordinate plot] (y - inches; x = weather types; day of week = colors)
- Identify if there is correlation between bike id , frequency of trip, and duration of trip. (does a lower bike id represent an older bike, therefore worse condition causing shorter trips) [corr scatterplot]

### Visualize Relationships Between Features and Prediction Class

TBD

### Features That Could Be Added

### Exceptional Work

Exceptional Visualizations
- Weather Map + Density vector for number of active bikers